## 1. The `@vectorize` decorator

Numba’s vectorize allows Python functions taking scalar input arguments to be used as NumPy ufuncs. Creating a traditional NumPy ufunc is not the most straightforward process and involves writing some C code. Numba makes this easy. Using the `vectorize()` decorator, Numba can compile a pure Python function into a ufunc that operates over NumPy arrays as fast as traditional ufuncs written in C.

Using `vectorize()`, you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs.

The vectorize() decorator has two modes of operation:  

+ **Eager, or decoration-time, compilation**: If you pass one or more type signatures to the decorator, you will be building a Numpy universal function (ufunc). The rest of this subsection describes building ufuncs using decoration-time compilation.
+ **Lazy, or call-time, compilation**: When not given any signatures, the decorator will give you a Numba dynamic universal function (DUFunc) that dynamically compiles a new kernel when called with a previously unsupported input type. A later subsection, “Dynamic universal functions”, describes this mode in more depth.  
As described above, if you pass a list of signatures to the `vectorize()` decorator, your function will be compiled into a Numpy ufunc. In the basic case, only one signature will be passed:

In [3]:
import numpy as np
from numba import vectorize, float64

@vectorize([float64(float64, float64)])
def f(x, y):
    return x + y

In [39]:
a = np.random.rand(1000).astype(np.float64)
b = np.random.rand(1000).astype(np.float64)

In [49]:
%timeit -r 10 -n 5 a + b

The slowest run took 52.55 times longer than the fastest. This could mean that an intermediate result is being cached.
17.4 µs ± 22.8 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [50]:
%timeit -r 10 -n 5 f(a, b)

The slowest run took 38.50 times longer than the fastest. This could mean that an intermediate result is being cached.
13.1 µs ± 16.8 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


If you pass several signatures, beware that you have to pass most specific signatures before least specific ones (e.g., single-precision floats before double-precision floats), otherwise type-based dispatching will not work as expected:

In [51]:
from numba import int32, int64, float32, float64


@vectorize([int32(int32, int32), int64(int64, int64), float32(float32, float32), float64(float64, float64)])
def g(x, y):
    return x + y

In [56]:
a = np.random.randint(low=0, high=100, size=1000).astype(np.int32)
b = np.random.randint(low=0, high=100, size=1000).astype(np.int32)

In [61]:
%timeit -r 10 -n 5 f(a, b)

The slowest run took 10.02 times longer than the fastest. This could mean that an intermediate result is being cached.
18.5 µs ± 16.8 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [62]:
%timeit -r 10 -n 5 g(a, b)

The slowest run took 38.21 times longer than the fastest. This could mean that an intermediate result is being cached.
6.55 µs ± 10.2 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [66]:
# will fail on other types:

a = np.linspace(0, 1 + 1j, 6)
a

array([0. +0.j , 0.2+0.2j, 0.4+0.4j, 0.6+0.6j, 0.8+0.8j, 1. +1.j ])

In [67]:
f(a, a)

TypeError: ufunc 'f' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [68]:
g(a, a)

TypeError: ufunc 'g' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

You might ask yourself, “why would I go through this instead of compiling a simple iteration loop using the `@jit decorator?`”. The answer is that NumPy ufuncs automatically get other features such as reduction, accumulation or broadcasting. Using the example above:

In [82]:
def g1(x, y):
    return x + y

print([attr for attr in dir(g) if attr not in dir(g1)])
# The most important methods are 'reduce' and 'accumulate'

['_DUFunc__base_kwargs', '__weakref__', '_add_loop', '_compile_for_args', '_compile_for_argtys', '_dispatcher', '_frozen', '_install_cg', '_install_type', '_keepalive', '_lower_me', '_type_me', 'accumulate', 'add', 'at', 'build_ufunc', 'disable_compile', 'find_ewise_function', 'identity', 'nargs', 'nin', 'nout', 'ntypes', 'outer', 'reduce', 'reduceat', 'targetoptions', 'types', 'ufunc']


In [79]:
a = np.arange(12).reshape(3, 4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [83]:
g.reduce(a, axis=0)

array([12, 15, 18, 21])

In [84]:
g.reduce(a, axis=1)

array([ 6, 22, 38])

In [85]:
g.accumulate(a, axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  6,  8, 10],
       [12, 15, 18, 21]])

In [86]:
g.accumulate(a, axis=1)

array([[ 0,  1,  3,  6],
       [ 4,  9, 15, 22],
       [ 8, 17, 27, 38]])

The `vectorize()` decorator supports multiple ufunc targets:  
  
Target | Description
- | -
cpu|Single threaded CPU
parallel|Multi-core CPU
cuda|CUDA GPU



A general guideline is to choose different targets for different data sizes and algorithms. 
+ The “cpu” target works well for small data sizes (approx. less than 1KB) and low compute intensity algorithms. It has the least amount of overhead. 
+ The “parallel” target works well for medium data sizes (approx. less than 1MB). Threading adds a small delay. 
+ The “cuda” target works well for big data sizes (approx. greater than 1MB) and high compute intensity algorithms. Transferring memory to and from the GPU adds significant overhead.

## 2. The `@guvectorize` decorator

While `vectorize()` allows you to write ufuncs that work on one element at a time, the `guvectorize()` decorator takes the concept one step further and allows you to write ufuncs that will work on an arbitrary number of elements of input arrays, and take and return arrays of differing dimensions. The typical example is a running median or a convolution filter.

Contrary to `vectorize()` functions, `guvectorize()` functions *don’t return their result value*: *they take it as an array argument, which must be filled in by the function.* This is because the array is actually allocated by NumPy’s dispatch mechanism, which calls into the Numba-generated code.

In [1]:
from numba import guvectorize, int64

@guvectorize([(int64[:], int64, int64[:])], "(n), () -> (n)", nopython=True)
def g(x, y, res):
    for i in range(len(x)):
        res[i] = x[i] + y    

The underlying Python function simply adds a given scalar (`y`) to all elements of a 1-dimension array. What’s more interesting is the declaration. There are two things there:

+ the declaration of input and output layouts, in symbolic form: `(n),()->(n)` tells NumPy that the function takes a *n-element one-dimension array, a scalar (symbolically denoted by the empty tuple ()) and returns a n-element one-dimension array*;
+ the list of supported concrete signatures as per `@vectorize`; here, as in the above example, we demonstrate int64 arrays.

In [4]:
a = np.random.randint(low=0, high=100, size=100).astype(np.int64)
b = 10

In [7]:
%timeit -r 10 -n 5 a + b

The slowest run took 10.03 times longer than the fastest. This could mean that an intermediate result is being cached.
2.29 µs ± 2.25 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [9]:
%timeit -r 10 -n 5 g(a, b)

1.58 µs ± 1 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [129]:
@guvectorize([(int64[:, :], int64, int64[:, :])], "(n, m), () -> (n, m)", nopython=True)
def g(x, y, res):
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            res[i, j] = x[i, j] + y  

In [123]:
a = np.arange(6).reshape(2, 3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [126]:
%timeit -r 10 -n 5 a + b

1.21 µs ± 804 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [127]:
%timeit -r 10 -n 5 g(a, b)  # slower

The slowest run took 7.03 times longer than the fastest. This could mean that an intermediate result is being cached.
7.1 µs ± 5.56 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


## 3. Dynamic universal functions

As described above, if you do not pass any signatures to the `vectorize()` decorator, your Python function will be used to build a dynamic universal function, or `DUFunc`. For example:

In [132]:
from numba import vectorize

@vectorize
def f(x, y):
    return x + y

The resulting `f()` is a DUFunc instance that starts with no supported input types. As you make calls to `f()`, Numba generates new kernels whenever you pass a previously unsupported input type. Given the example above, the following set of interpreter interactions illustrate how dynamic compilation works:

In [136]:
print(f)
print(f.ufunc)
print(f.ufunc.types)

<numba._DUFunc 'f'>
<ufunc 'f'>
[]


In [138]:
f(3, 4)
print(f)
print(f.ufunc)
print(f.ufunc.types)

<numba._DUFunc 'f'>
<ufunc 'f'>
['ll->q']


In [139]:
f(1., 2.)
print(f)
print(f.ufunc)
print(f.ufunc.types)  # Numba added a second loop for dealing with floating-point inputs, "dd->d".

<numba._DUFunc 'f'>
<ufunc 'f'>
['ll->q', 'dd->d']


In [140]:
f(1, 2.)
print(f)
print(f.ufunc)
print(f.ufunc.types)  #  Numpy ufunc casting rules are still in effect, not add new type.

<numba._DUFunc 'f'>
<ufunc 'f'>
['ll->q', 'dd->d']


If we had passed in floating-point arguments first, any calls with integer arguments would be cast to double-precision floating-point values. For example

In [142]:
@vectorize
def g(a, b):
    return a / b
print(g)
print(g.ufunc)
print(g.ufunc.types)

<numba._DUFunc 'g'>
<ufunc 'g'>
[]


In [144]:
g(2., 3.)  # double
print(g)
print(g.ufunc)
print(g.ufunc.types)

<numba._DUFunc 'g'>
<ufunc 'g'>
['dd->d']


In [145]:
g(2, 3)  # int
print(g)
print(g.ufunc)
print(g.ufunc.types)  # not add new type this time

<numba._DUFunc 'g'>
<ufunc 'g'>
['dd->d']
